In [1]:
!pip install torch_geometric

In [2]:
import numpy as np
import torch
from torch_geometric.data import Data
import gymnasium as gym
from gymnasium import spaces
import gerry_environment_13

#Inch Worm Test

In [3]:
def normalize_array(data,min_val, max_val):

  normal_array = (data - min_val) / (max_val - min_val)

  return np.round(normal_array,2)

In [4]:
import torch
import numpy as np

# ---- Setup initial data for inchworm ----
num_voters = 10
num_districts = 1
init_opinions = np.array([0,0,0,1,2,3,4,5,5,5], dtype=np.float32).reshape(num_voters, 1)

# trivial positions (not used in inchworm)
pos = np.zeros((num_voters, 2), dtype=np.float32)

# no base edges (inchworm works only via districting)
so_edge = np.empty((2,0),dtype=np.int64)
edge_attr  = np.empty((0,),dtype=np.float32)


# initial geographical (everyone in district 0 for now)
geographical_edge = np.zeros((num_voters, num_voters), dtype=np.float32)
geographical_edge = [
                      [0,0,0,1,0,0,0,0,0,0],
                      [0,0,0,1,0,0,0,0,0,0],
                      [0,0,0,1,0,0,0,0,0,0],
                      [1,1,1,0,1,0,0,1,1,1],
                      [0,0,0,1,0,1,0,0,1,1],
                      [0,0,0,0,1,0,1,0,0,0],
                      [0,0,0,0,0,1,0,1,0,0],
                      [0,0,0,1,0,0,1,0,1,0],
                      [0,0,0,1,1,0,0,1,0,1],
                      [0,0,0,1,1,0,0,0,1,0]
]

dist_label = np.zeros(num_voters, dtype=np.int32)

# no reps yet
reps = [-1]*num_districts

geographical_edge = []

# pack into FrankenData
init_data = gerry_environment_13.FrankenData(
    social_edge=so_edge,
    geographical_edge= geographical_edge,
    orig_edge_num=2,
    opinion=init_opinions,
    pos=pos,
    reps=reps,
    dist_label=dist_label,
    edge_attr=edge_attr
)


In [5]:
env = gerry_environment_13.FrankenmanderingEnv(num_voters=num_voters,
                          num_districts=num_districts,
                          FrankenData=init_data,
                                               )

# reset will now just deepcopy init_data
obs, _ = env.reset()
print("t=0:", obs.opinion.squeeze().tolist())


t=0: [0.0, 0.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 5.0, 5.0]


In [6]:
def run_inchworm(env, districts):
    history = []
    for t, district in enumerate(districts, 1):
        # Build assignment matrix
        assignment = np.zeros((env.num_voters, env.num_districts), dtype=np.float32)
        for v in range(env.num_voters):
            if v in district:
                assignment[v] = 1.0
            # else:
            #     assignment[v, 1] = 1.0

        obs, reward, terminated, truncated, info = env.step(assignment)

        opinions = obs.opinion.squeeze().tolist()
        reps = obs.reps.numpy().tolist()
        history.append((opinions, reps))

        print(f"t={t}, reps={reps}, opinions={opinions}")
    return history

In [7]:
districts = [
    [2,3,9],
    [1,3,8],
    [0,3,7],
    [3,4,9],
    [4,5,8],
    [5,6,7],
    [6,7,8],
    [7,8,9]
]

history = run_inchworm(env, districts)

t=1, reps=[3], opinions=[0.0, 0.0, 1.0, 1.0, 2.0, 3.0, 4.0, 5.0, 5.0, 6.0]
t=2, reps=[3], opinions=[0.0, 1.0, 1.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 6.0]
t=3, reps=[3], opinions=[1.0, 1.0, 1.0, 1.0, 2.0, 3.0, 4.0, 6.0, 6.0, 6.0]
t=4, reps=[4], opinions=[1.0, 1.0, 1.0, 2.0, 2.0, 3.0, 4.0, 6.0, 6.0, 7.0]
t=5, reps=[5], opinions=[1.0, 1.0, 1.0, 2.0, 3.0, 3.0, 4.0, 6.0, 7.0, 7.0]
t=6, reps=[6], opinions=[1.0, 1.0, 1.0, 2.0, 3.0, 4.0, 4.0, 5.0, 7.0, 7.0]
t=7, reps=[7], opinions=[1.0, 1.0, 1.0, 2.0, 3.0, 4.0, 5.0, 5.0, 6.0, 7.0]
t=8, reps=[8], opinions=[1.0, 1.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 6.0, 6.0]


/content/gerry_environment_13.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.pos = torch.tensor(pos, dtype=torch.float32)
/content/gerry_environment_13.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.geographical_edge = torch.tensor(geographical_edge, dtype=torch.long)


In [8]:
# import numpy as np
# simple test
# def update_opinion(voter_opinion, rep_opinion):
#     """Apply Inchworm update rule for a single voter."""
#     diff = rep_opinion - voter_opinion
#     if abs(diff) < 3:
#         return voter_opinion + np.sign(diff)
#     else:
#         return voter_opinion - np.sign(diff)

# def elect_representative(district, opinions):
#     """Median voter becomes representative."""
#     members = sorted(district, key=lambda i: opinions[i])
#     median_index = members[len(members) // 2]
#     return median_index

# def step_inchworm(opinions, district):
#     """Run one step of Inchworm given a district."""
#     rep = elect_representative(district, opinions)
#     rep_opinion = opinions[rep]
#     new_opinions = opinions.copy()

#     for v in district:
#         if v == rep:
#             continue  # rep stays unchanged
#         new_opinions[v] = update_opinion(opinions[v], rep_opinion)

#     return new_opinions, rep



# for t, d in enumerate(districts, 1):
#     opinions, rep = step_inchworm(opinions, d)
#     print(f"t={t}, rep={rep}, opinions={opinions}")

# res : t=0: [0 0 0 1 2 3 4 5 5 5]
# t=1, rep=3, opinions=[0 0 1 1 2 3 4 5 5 6]
# t=2, rep=3, opinions=[0 1 1 1 2 3 4 5 6 6]
# t=3, rep=3, opinions=[1 1 1 1 2 3 4 6 6 6]
# t=4, rep=4, opinions=[1 1 1 2 2 3 4 6 6 7]
# t=5, rep=5, opinions=[1 1 1 2 3 3 4 6 7 7]
# t=6, rep=6, opinions=[1 1 1 2 3 4 4 5 7 7]
# t=7, rep=7, opinions=[1 1 1 2 3 4 5 5 6 7]
# t=8, rep=8, opinions=[1 1 1 2 3 4 5 6 6 6]
